<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C1-Basic_Networking_Tools/TCP_Proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a TCP Proxy

There are several reasons to have a __TCP__ proxy in your tool belt. You might use one for forwarding traffic to bounce from host to host, or when assessing network based software. When performing penetration tests in enterprise environments, you probably won't be able to run __Wireshark__; nor will you be able to load drivers to sniff the loopback on Windows, and network segmentation will prevent you from running your tools directly against your target host. We've built simple Python proxies, like this one, in various cases to help you understand unknown protocols, modify traffic being sent to an application, and create test cases for fuzzers. 

The proxy has a few moving parts. Let's summarize the four main functions we need to write. We need to display the communication between the local and remote machines to the console (__hexdump__). We need to receive data from an incoming socket from either the local or remote machine (__receive_from__). We need to manage the traffic direction between remote and local machines (__proxy_handler__). Finally, we need to set up a listening socket and pass it to our __proxy_handler__ (__server_loop__).

Let's get to it. Open a new file called __proxy.py__:

In [ ]:
import sys
import socket
import threading

HEX_FILTER = ''.join(
    [len(repr(chr(i))) == 3) and chr(i) or '.' for i in range(256)]) #1

def hexdump(src, length=16, show=True):
    if isinstance(src, bytes): #2
        src = src.decode()

    results = list()
    for i in range(0, len(src), length):
        word = str(src[i:i+length]) #3

        printable = word.translate(HEX_FILTER)
        hexa = ' '.join([f'{ord(c):02X}' for c in word])
        hexwidth = length * 3
        results.append(f'{i:04x}  {hexa:<{hexwidth}}  {printable}')
    if show:
        for line in results:
            print(line)
    else:
        return results